In [2]:
import sys
import os
from json import load, dump

import pandas as pd
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show

In [3]:
df_original = pd.read_csv('../data/preprocessed.csv')

In [4]:
df_original.head()

,TYPE_WACHTTIJD,WACHTTIJD,OMS_WACHTTIJD,SPECIALISME,ROAZ_REGIO,NAAM_INSTELLING,TYPE_ZORGINSTELLING
0,Behandeling,90.0,Tonsillectomie en/of adenotomie (kno),KNO (302),Noord-NL,Wilhelmina Ziekenhuis,Ziekenhuis
1,Polikliniekbezoek,21.0,Orthopedie - Heupklachten,Orthopedie (305),Oost,Stichting Rijnstate Ziekenhuis,Ziekenhuis
2,Behandeling,166.0,Initiële staaroperatie (oogheelkunde),Oogheelkunde (301),SpzNet AMC,Stichting Dijklander Ziekenhuis,Ziekenhuis
3,Behandeling,14.0,Triggerfingerrelease (plastische chirurgie),Plastische Chirurgie (304),Brabant,Xpert Clinics Hand- en polszorg B.V.,Kliniek
4,Polikliniekbezoek,28.0,Chirurgie (heelkunde) - Proctologie,Chirurgie (heelkunde) (303),Noord-NL,Ziekenhuis Nij Smellinghe,Ziekenhuis


In [5]:
df_train, df_test = train_test_split(df_original, test_size=0.2)

df_train_output = df_train[['WACHTTIJD']]
df_train_inputs = df_train.drop(['WACHTTIJD','NAAM_INSTELLING'], axis=1)

df_test_output = df_test[['WACHTTIJD']]
df_test_inputs = df_test.drop(['WACHTTIJD','NAAM_INSTELLING'], axis=1)

In [6]:
model = ExplainableBoostingRegressor(feature_names=df_train_inputs.columns)
model.fit(df_train_inputs.to_numpy(), df_train_output.to_numpy())
model.score(df_test_inputs.to_numpy(), df_test_output.to_numpy())

0.3479381371921183

In [7]:
ebm_global = model.explain_global()
ebm_local = model.explain_local(df_test_inputs.to_numpy(), df_test_output.to_numpy())

show([ebm_global, ebm_local])

c:\Users\willemm\AppData\Local\Programs\Python\Python39\lib\site-packages\interpret\visual\udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
c:\Users\willemm\AppData\Local\Programs\Python\Python39\lib\site-packages\interpret\visual\udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
c:\Users\willemm\AppData\Local\Programs\Python\Python39\lib\site-packages\interpret\visual\udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/2721651267328/ -->
 Open in new window

In [14]:
model_data = get_model_data(model)
sample_data = get_sample_data(model, df_test_inputs.to_numpy(), df_test_output.to_numpy())

dump(model_data, open('./model.json', 'w'))
dump(sample_data, open('./sample.json', 'w'))